In [182]:
# CURRENTLY, goal df includes....
# each teams season stats summary 
#  - standard game stats (DONE)
#  - wins (DONE)
#  - losses (DONE)
#  - record vs
#    - ranked (top N)
#    - common opponents (or other conferences)
#    - each other
#  - team ratings
#  - encode conferences w/ avg ratings

import pandas as pd
pd.set_option('display.max_columns', 500)

# --------------------------------------------------------------------------------------------------------------------

def LLoc(loc):
    # return opposite of WLoc for L team
    key = {'H': 'A', 'A': 'H', 'N': 'N'}
    return key[loc]
    
def split_rows(df, w_map, l_map):
    df1 = df.copy()
    df2 = df.copy()
    
    to_keep = ['Season', 'NumOT']
    df1.drop(columns=[x for x in l_map.keys() if x not in to_keep], inplace=True)
    df2.drop(columns=[x for x in w_map.keys() if x not in to_keep], inplace=True)
    
    df1.rename(columns=w_map, inplace=True)
    df2.rename(columns=l_map, inplace=True)
    
    # win-loss count
    df1["W"], df1["L"] = 1, 0
    df2["W"], df2["L"] = 0, 1
    
    return pd.concat([df1, df2]).sort_index()

def add_conferences(df):
    conf = pd.read_csv("data/MTeamConferences.csv")
    team = pd.read_csv("data/MTeams.csv")
    team_and_conf = conf.merge(team)
    return team_and_conf[["Season", "TeamID", "TeamName", "ConfAbbrev"]].merge(df)

def add_column(df, after, name, values):
    df        = df.copy()
    col_index = df.columns.get_loc(after) + 1
    df.insert(col_index, name, values)
    return df

def add_percentages(df):
    df = add_column(df=df, after="FGA", name="FG%", values=df["FGM"] / df["FGA"])
    df = add_column(df=df, after="FGA3", name="FG3%", values=df["FGM3"] / df["FGA3"])
    df = add_column(df=df, after="FTA", name="FT%", values=df["FTM"] / df["FTA"])
    return df
    
def add_ratings():
    # average rating, max rating, min rating, last rating
    pass
 
def get_head_to_head_record(season, team1_id, team2_id):
    results  = pd.read_csv("data/MRegularSeasonCompactResults.csv")
    matchups = results.loc[
        (results['Season'] == season) & 
        (results['WTeamID'].isin([team1_id, team2_id])) & 
        (results['LTeamID'].isin([team1_id, team2_id]))
    ]
    
    counts = matchups[['WTeamID','LTeamID']].apply(pd.value_counts).fillna(0)
    return {
        team1_id: {"W": counts['WTeamID'][team1_id], "L": counts['LTeamID'][team1_id]}, 
        team2_id: {"W": counts['WTeamID'][team2_id], "L": counts['LTeamID'][team2_id]},
    }
    
def get_common_opponent_record(season, team1_id, team2_id):
    results  = pd.read_csv("data/MRegularSeasonCompactResults.csv")
    team1_matchups = results.loc[
        (results['Season'] == season) & 
        ((results['WTeamID'] == team1_id) | (results['LTeamID'] == team1_id)) 
    ]
    team2_matchups = results.loc[
        (results['Season'] == season) & 
        ((results['WTeamID'] == team2_id) | (results['LTeamID'] == team2_id)) 
    ]
    
    team1_opponents = set(list(team1_matchups['WTeamID'].unique()) + list(team1_matchups['LTeamID'].unique()))
    team2_opponents = set(list(team2_matchups['WTeamID'].unique()) + list(team2_matchups['LTeamID'].unique()))
    
    # remove self
    team1_opponents.remove(team1_id)
    team2_opponents.remove(team2_id)

    common_opponents = list(team1_opponents.intersection(team2_opponents))
    print(common_opponents)
    # keep only common matchups
    team1_matchups = team1_matchups.loc[
        (results['WTeamID'].isin(common_opponents)) | (results['LTeamID'].isin(common_opponents))
    ]
    team2_matchups = team2_matchups.loc[
        (results['WTeamID'].isin(common_opponents)) | (results['LTeamID'].isin(common_opponents))
    ]
    
    team1_counts = team1_matchups[['WTeamID','LTeamID']].apply(pd.value_counts).fillna(0)
    team2_counts = team2_matchups[['WTeamID','LTeamID']].apply(pd.value_counts).fillna(0)

    return {
        team1_id: {"W": team1_counts['WTeamID'][team1_id], "L": team1_counts['LTeamID'][team1_id]}, 
        team2_id: {"W": team2_counts['WTeamID'][team2_id], "L": team2_counts['LTeamID'][team2_id]},
    }

def get_ranked_oppenent_record():
    # ranked how high?
    pass
    

In [32]:
conf = pd.read_csv("data/MTeamConferences.csv")
team = pd.read_csv("data/MTeams.csv")
conf.merge(team)
# combine MTeamConferences and MMasseyOrdinals to get a df that 
# has this df (below) with average rating, max rating, min rating, last rating 
# for that season

,Season,TeamID,ConfAbbrev,TeamName,FirstD1Season,LastD1Season
0,1985,1102,wac,Air Force,1985,2021
1,1986,1102,wac,Air Force,1985,2021
2,1987,1102,wac,Air Force,1985,2021
3,1988,1102,wac,Air Force,1985,2021
4,1989,1102,wac,Air Force,1985,2021
...,...,...,...,...,...,...
11936,2021,1467,nec,Merrimack,2020,2021
11937,2021,1468,a_sun,Bellarmine,2021,2021
11938,2021,1469,wac,Dixie St,2021,2021
11939,2021,1470,wac,Tarleton St,2021,2021


In [38]:
ratings = pd.read_csv("data/MMasseyOrdinals.csv")
ratings["SystemName"].value_counts()

SAG    111701
MOR    110711
POM    108220
DOK     94974
WLK     90347
        ...  
PMC       351
HRN       351
CRW       351
BP5       345
PH        326
Name: SystemName, Length: 179, dtype: int64

In [27]:
box_scores = pd.read_csv("data/MRegularSeasonDetailedResults.csv")

box_scores = add_column(df=box_scores, after="WLoc", name="LLoc", values=box_scores["WLoc"].map(LLoc))
box_scores = add_column(df=box_scores, after="WDR", name="WTR", values=box_scores["WOR"]+box_scores["WDR"])
box_scores = add_column(df=box_scores, after="LDR", name="LTR", values=box_scores["LOR"]+box_scores["LDR"])

# map from old columns to new_columns
w_map = {
    'Season': 'Season', 'DayNum': 'DayNum', 'WTeamID': 'TeamID', 'WScore': 'Score', 'WLoc': 'Loc', 'NumOT': 'NumOT', 
    'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA', 'WOR': 'OR', 
    'WDR': 'DR', 'WTR': 'TR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
}
l_map = {
    'Season': 'Season', 'DayNum': 'DayNum', 'LTeamID': 'TeamID', 'LScore': 'Score', 'LLoc': 'Loc', 'NumOT': 'NumOT',
    'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA', 'LOR': 'OR', 
    'LDR': 'DR', 'LTR': 'TR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
}


detailed_results_GAME = split_rows(box_scores.copy(), w_map, l_map)
detailed_results_SZN_AVG = detailed_results_GAME.groupby(['Season', 'TeamID'], as_index=False).mean()

In [106]:
detailed_results_GAME

,Season,TeamID,Score,Loc,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,TR,Ast,TO,Stl,Blk,PF,W,L
0,2003,1104,68,N,0,27,58,3,14,11,18,14,24,38,13,23,7,1,22,1,0
0,2003,1328,62,N,0,22,53,2,10,16,22,10,22,32,8,18,9,2,20,0,1
1,2003,1393,63,N,0,24,67,6,24,9,20,20,25,45,7,12,8,6,16,0,1
1,2003,1272,70,N,0,26,62,8,20,10,19,15,28,43,16,13,4,4,18,1,0
2,2003,1266,73,N,0,24,58,8,18,17,29,17,26,43,15,10,5,2,25,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92829,2020,1402,85,H,0,34,57,4,12,13,16,6,25,31,20,14,9,0,24,1,0
92830,2020,1427,69,N,0,26,67,10,27,7,7,12,22,34,11,13,4,2,18,0,1
92830,2020,1412,74,N,0,26,53,8,14,14,20,8,29,37,12,16,6,5,16,1,0
92831,2020,1450,82,N,0,27,55,10,21,18,25,6,31,37,16,13,9,3,24,1,0


In [102]:
detailed_results_SZN_AVG

,Season,TeamID,Score,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,TR,Ast,TO,Stl,Blk,PF,W,L
0,2003,1102,57.250000,0.000000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,21.000000,13.000000,11.428571,5.964286,1.785714,18.750000,0.428571,0.571429
1,2003,1103,78.777778,0.296296,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,29.703704,15.222222,12.629630,7.259259,2.333333,19.851852,0.481481,0.518519
2,2003,1104,69.285714,0.035714,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,37.500000,12.107143,13.285714,6.607143,3.785714,18.035714,0.607143,0.392857
3,2003,1105,71.769231,0.153846,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,36.615385,14.538462,18.653846,9.307692,2.076923,20.230769,0.269231,0.730769
4,2003,1106,63.607143,0.035714,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,36.142857,11.678571,17.035714,8.357143,3.142857,18.178571,0.464286,0.535714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6182,2020,1463,74.535714,0.285714,26.392857,57.285714,8.892857,24.071429,12.857143,17.892857,8.607143,29.357143,37.964286,14.857143,13.000000,6.214286,3.285714,15.035714,0.750000,0.250000
6183,2020,1464,71.193548,0.096774,25.580645,61.548387,7.193548,23.064516,12.838710,18.451613,13.709677,25.032258,38.741935,11.967742,12.225806,5.322581,2.516129,17.806452,0.516129,0.483871
6184,2020,1465,76.222222,0.111111,25.296296,58.925926,9.259259,24.555556,16.370370,20.777778,11.037037,28.481481,39.518519,12.777778,12.518519,3.592593,3.518519,18.296296,0.629630,0.370370
6185,2020,1466,67.464286,0.035714,22.892857,54.678571,6.642857,19.821429,15.035714,21.714286,9.785714,25.714286,35.500000,10.392857,15.035714,6.142857,2.250000,20.214286,0.392857,0.607143


In [31]:
add_conferences(add_percentages(detailed_results_SZN_AVG))

,Season,TeamID,TeamName,ConfAbbrev,Score,NumOT,FGM,FGA,FG%,FGM3,FGA3,FG3%,FTM,FTA,FT%,OR,DR,TR,Ast,TO,Stl,Blk,PF,W,L
0,2003,1102,Air Force,mwc,57.250000,0.000000,19.142857,39.785714,0.481149,7.821429,20.821429,0.375643,11.142857,17.107143,0.651357,4.178571,16.821429,21.000000,13.000000,11.428571,5.964286,1.785714,18.750000,0.428571,0.571429
1,2004,1102,Air Force,mwc,60.178571,0.000000,20.285714,42.035714,0.482583,8.464286,22.214286,0.381029,11.142857,15.714286,0.709091,6.142857,15.357143,21.500000,13.250000,10.785714,7.785714,2.357143,16.642857,0.785714,0.214286
2,2005,1102,Air Force,mwc,61.241379,0.068966,21.413793,47.724138,0.448699,8.896552,24.413793,0.364407,9.517241,13.068966,0.728232,7.620690,15.413793,23.034483,13.655172,10.068966,8.793103,1.724138,16.482759,0.586207,0.413793
3,2006,1102,Air Force,mwc,63.500000,0.000000,21.928571,45.571429,0.481191,8.607143,21.678571,0.397035,11.035714,14.785714,0.746377,6.785714,17.785714,24.571429,14.178571,10.928571,8.000000,2.107143,15.392857,0.785714,0.214286
4,2007,1102,Air Force,mwc,68.500000,0.000000,22.800000,46.933333,0.485795,9.066667,22.466667,0.403561,13.833333,18.200000,0.760073,6.466667,20.866667,27.333333,15.033333,10.166667,6.266667,1.466667,15.166667,0.733333,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6182,2019,1465,Cal Baptist,wac,75.461538,0.000000,26.038462,59.038462,0.441042,8.807692,25.230769,0.349085,14.576923,18.961538,0.768763,10.076923,26.269231,36.346154,10.615385,12.500000,4.346154,2.615385,21.307692,0.461538,0.538462
6183,2020,1465,Cal Baptist,wac,76.222222,0.111111,25.296296,58.925926,0.429290,9.259259,24.555556,0.377074,16.370370,20.777778,0.787879,11.037037,28.481481,39.518519,12.777778,12.518519,3.592593,3.518519,18.296296,0.629630,0.370370
6184,2019,1466,North Alabama,a_sun,65.620690,0.000000,22.931034,59.655172,0.384393,7.241379,24.344828,0.297450,12.517241,18.379310,0.681051,10.862069,25.103448,35.965517,10.172414,13.655172,6.275862,2.137931,18.965517,0.241379,0.758621
6185,2020,1466,North Alabama,a_sun,67.464286,0.035714,22.892857,54.678571,0.418681,6.642857,19.821429,0.335135,15.035714,21.714286,0.692434,9.785714,25.714286,35.500000,10.392857,15.035714,6.142857,2.250000,20.214286,0.392857,0.607143


chalk = chalk_submission() / "file_path"
upset = upset_submission() / "file_path"

key = ""
print(gsheet_bracket_from_submission(chalk, key))
print(gsheet_bracket_from_submission(upset, key))

